In [3]:
import pandas as pd
import os

Para poder seleccionar bien las variables a utilizar, creo un dataframe mucho mas chico para no tener que procesar la totalidad de los datos.

In [4]:
df = pd.read_parquet('datos-todas-estaciones.parquet')
df.columns = [col.lower().strip() for col in df.columns]

df_mini = df.copy()[:1000]

### Selección de variables.

In [5]:
features = (df.isna().sum() * 100 / len(df)).reset_index()
features.columns = ['columna', 'percentNa']

In [6]:
features.sort_values('percentNa').query('percentNa <= 65')

,columna,percentNa
0,fecha,0.000000
30,id_estacion,0.000000
3,temperatura_abrigo_150cm_minima,6.066566
2,temperatura_abrigo_150cm_maxima,6.360656
1,temperatura_abrigo_150cm,6.553649
11,precipitacion_pluviometrica,7.020599
18,humedad_media_8_14_20,9.662863
16,tesion_vapor_media,10.929604
19,rocio_medio,12.666109
28,horas_frio,14.436710


In [ ]:
cols = features.sort_values(by='percentNa').reset_index(drop=True)[:20].columna.values
cols

array(['fecha', 'id_estacion', 'temperatura_abrigo_150cm_minima',
       'temperatura_abrigo_150cm_maxima', 'temperatura_abrigo_150cm',
       'precipitacion_pluviometrica', 'humedad_media_8_14_20',
       'tesion_vapor_media', 'rocio_medio', 'horas_frio',
       'velocidad_viento_200cm_media', 'velocidad_viento_1000cm_media',
       'radiacion_global', 'heliofania_efectiva', 'heliofania_relativa',
       'direccion_viento_1000cm', 'direccion_viento_200cm',
       'evapotranspiracion_potencial', 'humedad_media',
       'temperatura_suelo_10cm_media'], dtype=object)

In [18]:
unidades = {'fecha': 'fecha',
 'temperatura_abrigo_150cm': '°C',
 'temperatura_abrigo_150cm_maxima': '°C',
 'temperatura_abrigo_150cm_minima': '°C',
 'temperatura_intemperie_5cm_minima': '°C',
 'temperatura_intemperie_50cm_minima': '°C',
 'temperatura_suelo_5cm_media': '°C',
 'temperatura_suelo_10cm_media': '°C',
 'temperatura_inte_5cm': '°C',
 'temperatura_intemperie_150cm_minima': '°C',
 'humedad_suelo': '%',
 'precipitacion_pluviometrica': 'mm',
 'precipitacion_cronologica': 'mm',
 'precipitacion_maxima_30minutos': 'mm',
 'heliofania_efectiva': 'horas',
 'heliofania_relativa': '%',
 'tesion_vapor_media': 'hPa',
 'humedad_media': '%',
 'humedad_media_8_14_20': '%',
 'rocio_medio': '°C',
 'duracion_follaje_mojado': 'horas',
 'velocidad_viento_200cm_media': 'm/s',
 'direccion_viento_200cm': 'grados',
 'velocidad_viento_1000cm_media': 'm/s',
 'direccion_viento_1000cm': 'grados',
 'velocidad_viento_maxima': 'm/s',
 'presion_media': 'hPa',
 'radiacion_global': 'MJ/m²',
 'horas_frio': 'horas',
 'unidades_frio': 'unidades'}

unidades_completas = {
    "fecha": "Fecha (día, mes, año, hora)",
    "°C": "Grados Celsius (temperatura)",
    "%": "Porcentaje (%)",
    "mm": "Milímetros (precipitación)",
    "horas": "Horas",
    "hPa": "Hectopascales (presión o tensión de vapor)",
    "m/s": "Metros por segundo (velocidad del viento)",
    "grados": "Grados (dirección del viento, 0° a 360°)",
    "MJ/m²": "Megajulios por metro cuadrado (radiación solar)",
    "unidades": "Unidades de frío (índice agroclimático)"
}


## Valores nulos.

In [36]:
pd.set_option("display.max_columns", 200)

In [43]:
df = df[cols]
df.isna().sum()

fecha                                   0
id_estacion                             0
temperatura_abrigo_150cm_minima     56047
temperatura_abrigo_150cm_maxima     58764
temperatura_abrigo_150cm            60547
precipitacion_pluviometrica         64861
humedad_media_8_14_20               89272
tesion_vapor_media                 100975
rocio_medio                        117018
horas_frio                         133376
velocidad_viento_200cm_media       350221
velocidad_viento_1000cm_media      350760
radiacion_global                   419849
heliofania_efectiva                420280
heliofania_relativa                427929
direccion_viento_1000cm            444666
direccion_viento_200cm             501034
evapotranspiracion_potencial       517424
humedad_media                      539444
temperatura_suelo_10cm_media       540783
dtype: int64

In [45]:
df

,fecha,id_estacion,temperatura_abrigo_150cm_minima,temperatura_abrigo_150cm_maxima,temperatura_abrigo_150cm,precipitacion_pluviometrica,humedad_media_8_14_20,tesion_vapor_media,rocio_medio,horas_frio,velocidad_viento_200cm_media,velocidad_viento_1000cm_media,radiacion_global,heliofania_efectiva,heliofania_relativa,direccion_viento_1000cm,direccion_viento_200cm,evapotranspiracion_potencial,humedad_media,temperatura_suelo_10cm_media
0,2009-08-06 00:00:00.0,A872801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,NaN,NaN,NaN
1,2009-08-07 00:00:00.0,A872801,7.3,12.3,9.572915,0.0,81.0,9.712307,6.544751,1.494000,NaN,NaN,NaN,NaN,NaN,,,NaN,83.0,9.702779
2,2009-08-08 00:00:00.0,A872801,1.7,17.4,8.314584,0.0,77.0,8.052779,3.762041,13.612010,NaN,NaN,NaN,NaN,NaN,,,NaN,81.0,9.373611
3,2009-08-09 00:00:00.0,A872801,0.8,17.1,7.433333,0.0,73.0,7.681013,3.135792,13.944010,NaN,NaN,NaN,NaN,NaN,,,NaN,77.0,8.793056
4,2009-08-10 00:00:00.0,A872801,3.9,22.2,11.295140,0.0,52.0,7.810816,3.396338,7.801996,NaN,NaN,NaN,NaN,NaN,,,NaN,58.0,8.979861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923862,2025-09-07 00:00:00.0,NH0550,1.5,23.9,12.700000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.883305,5.2,44.52,None,None,NaN,NaN,NaN
923863,2025-09-08 00:00:00.0,NH0550,6.3,29.2,17.750000,0.0,NaN,NaN,NaN,NaN,1.984584,2.480729,19.839062,10.0,85.40,None,None,NaN,NaN,NaN
923864,2025-09-09 00:00:00.0,NH0550,4.5,29.5,17.000000,0.0,43.0,10.844036,8.069238,NaN,6.966665,8.708331,19.952995,10.0,85.25,N,None,3.399957,NaN,21.699999
923865,2025-09-10 00:00:00.0,NH0550,10.4,35.5,22.950000,0.0,NaN,NaN,NaN,NaN,4.965418,6.206773,19.330389,9.5,80.78,None,None,NaN,NaN,NaN
